In [36]:
#Importing the libraries

from torch import nn, randn, tensor, optim, argmax, no_grad, save
import torch.nn.functional as F
import numpy as np
from cv2 import imread
import cv2
import os
import matplotlib.pyplot as plt
from tqdm import tqdm


In [2]:
#Reading the images
class preprocess():
    data_list = []
    catcount = 0
    dogcount = 0

    rootdir = ('../Datasets/kagglecatsanddogs_3367a/')
    Cats = "../Datasets/kagglecatsanddogs_3367a/PetImages/Cat"
    Dogs = "../Datasets/kagglecatsanddogs_3367a/PetImages/Dog"
    cat_dir = {Cats:0, Dogs:1}

    def get_data(self):    
        for labels in self.cat_dir:
            print(labels)
            for img in tqdm(os.listdir(labels)):
                try:
                    img_path = os.path.join(labels, img)
                    img = imread(img_path, cv2.IMREAD_GRAYSCALE) #Converting the image to numpy array
                    img = cv2.resize(img,(50,50)) #Resizing the image to 50*50
                    self.data_list.append([np.array(img), np.eye(2)[self.cat_dir[labels]]]) #Appending the image data and label into list

                    if labels == self.Cats:
                        self.catcount+=1    
                    else:
                        self.dogcount+=1
                except Exception as e: 
                    pass


        print('No of cats: ', self.catcount)
        print('No of dogs: ', self.dogcount)
        
        if self.data_list is None:
            print('Empty list')
            
        else:
            np.random.shuffle(self.data_list)  #Shuffle the numpy indices
            np.save('training_data.npy', self.data_list) #Save data into memory



In [3]:
#Only gets data in case there is no training_data.npy file is present, else use it from memory

if(os.path.isfile('training_data.npy') == True):
    data = np.load('training_data.npy', allow_pickle=True)
    
else:
    preprocess = preprocess()
    preprocess.get_data()

In [4]:
#Building the neural network

class net(nn.Module):
    def __init__(self):
        super().__init__()
        self.cnn1 = nn.Conv2d(1, 32, 5)
        self.cnn2 = nn.Conv2d(32, 64, 5) #(in_channels=32, out_channels=64, stride=1, kernel_size=5)
        self.cnn3 = nn.Conv2d(64, 128, 5) #(in_channels=64, out_channels=128, stride=1, kernel_size=5)
        
        x = randn(50*50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)
        
        self.fc1 = nn.Linear(in_features=self._to_linear, out_features=512)
        self.fc2 = nn.Linear(in_features=512, out_features=2)
    
    def convs(self,x):
        
        x = F.max_pool2d(F.relu(self.cnn1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.cnn2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.cnn3(x)), (2,2))
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x

    def forward(self,x):
        x = self.convs(x)
        x=x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x,dim=1)
    
net = net()
print(net)

net(
  (cnn1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (cnn2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (cnn3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=512, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=2, bias=True)
)


In [5]:
#Importing the optimizer
optimizer = optim.Adam(net.parameters(), lr = 0.001) #Declaring the optimizer
loss_functions = nn.MSELoss()


#Seperating X's and y's in our dataset

X = tensor([i[0] for i in data]).view(-1,50,50)
X = X/255.0 #Normalizing the images as preprocessing
y = tensor([i[1] for i in data])


#Splitting the dataset into training and testing

VAL_PER = 0.1
valid_sample = int(len(X)*VAL_PER)
print('Size of the validation sample is: ', valid_sample)

X_train = X[:-valid_sample]
y_train = y[:-valid_sample]

# total_length = (len(data) - valid_sample)

# testing_per = 0.2
# test_sample = int(total_length*testing_per)

X_test = X[-valid_sample:]
y_test = y[-valid_sample:]

# train = len(data) - total_length
print('Test set: ',len(X_test), ' Training set: ', len(X_train))


Size of the validation sample is:  2494
Test set:  2494  Training set:  22452


In [6]:
X_train.shape

torch.Size([22452, 50, 50])

In [9]:
EPOCHS = 3
BATCH_SIZE = 32
loss = None
for j in range(EPOCHS):

    for i in tqdm(range(0, len(X_train), BATCH_SIZE)):
        batch_X = X_train[i:BATCH_SIZE+i].view(-1,1,50,50)
        batch_y = (y_train[i:BATCH_SIZE+i]).double()
        
        net.zero_grad() #Why Zeeo grad?
        output = net(batch_X).double()
        loss = loss_functions(output, batch_y)
        loss.backward()
        optimizer.step()
        
print('Overall loss of the model is: ', loss)        
 

100%|████████████████████████████████████████████████████████████████████████████████| 702/702 [02:23<00:00,  4.90it/s]


Overall loss of the model is:  tensor(0.1385, dtype=torch.float64, grad_fn=<MseLossBackward>)


In [39]:
save(net.state_dict(),'model.h5')

In [35]:
#Accuracy metrics
total = 0
correct = 0
with no_grad():
    y_pred_test = (net(X_test.view(-1,1,50,50)))

    for i in tqdm(range(len(X_test))):
        total = total + 1
        if (argmax(y_pred_test[i])) == (argmax(y_test[i])):
            correct = correct +1

print('Overall accuracy of the model is: ', correct/total*100)

100%|███████████████████████████████████████████████████████████████████████████| 2494/2494 [00:00<00:00, 13717.28it/s]

Overall accuracy of the model is:  78.1074578989575
